#<p><center> **COMMONSENSE KNOWLEDGE BASES: MERONYMIC RELATIONSHIPS ACQUISITION AND THEIR VERIFICATION WITH LARGE LANGUAGE MODELS**

---

<p><center> Master thesis - MS in Language Analysis and Proccessing
<p><center> Julia Fidalgo Mariño
<p><center> Supervised by German Rigau</center></p>




---


#<p><center> **LANGUAGE MODELS: EXAMPLE WITH LLAMA 3.1**

---



###**QUESTIONS FOR THE FIRST TESTING**

**Question 1:** Is there always a meronymic relationship between “bird” and “tail”?

**Question 2:** Taking into account that there exist three types of meronymic relationships (“Part of”, “Member of” and “Substance of”), what type of meronymic relationship exists between “bird” and “tail”?

**Question 3:** Taking into account that “bird” and “tail” are not always meronymically related, provide a dictionary definition of “tail” in which this relation does not exist.

In [3]:
%%capture
!pip install -U accelerate transformers
!pip install bitsandbytes
!pip install torch

In [4]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import display, Markdown

In [7]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
#model_id = "microsoft/Phi-3-small-8k-instruct"

In [8]:
pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    # The quantization line
    model_kwargs={"torch_dtype": torch.bfloat16, "load_in_4bit": True}
)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [9]:
def query_model(
        system_message,
        user_message,
        temperature=0.1,
        max_length=256
        ):
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
        )
    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        #num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=pipeline.model.config.eos_token_id
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']

    return user_message + " " + answer  + " "

In [10]:
system_message = """
you are a linguist expert in Semantics and Lexicography, you always provide truthful information and you are consistent and coherent with your answers and definitions
"""

In [11]:
def colorize_text(text):
    for word, color in zip(["Question", "Answer"], ["blue", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [12]:
user_message="Is there always a meronymic relationship between “bird” and “tail”?"

In [13]:
response = query_model(
    system_message,
    user_message=user_message,
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='blue'>Question:</font>** Is there always a meronymic relationship between “bird” and “tail”? 

**<font color='green'>Answer:</font>** What a delightful question!

In linguistics, a meronymic relationship is a type of semantic relationship between two words, where one word is a part of the other. In this case, the question asks if there is a meronymic relationship between "bird" and "tail".

The answer is: No, there is no meronymic relationship between "bird" and "tail".

A meronymic relationship implies that one word is a part of the other, such as "arm" and "hand", where an arm is a part of a hand. In the case of "bird" and "tail", a bird does not have a tail as a part of it. Instead, a bird has a tail as a distinct feature or appendage.

So, while birds do have tails, the relationship between the two words is not meronymic. 

In [14]:
response2 = query_model(
    system_message,
    user_message="Taking into account that “bird” and “tail” are meronymically related, provide a dictionary definition of “tail” in which this relation does not exist",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response2)))



**<font color='blue'>Question:</font>** Taking into account that “bird” and “tail” are meronymically related, provide a dictionary definition of “tail” in which this relation does not exist 

**<font color='green'>Answer:</font>** What an intriguing question!

As a linguist expert in Semantics and Lexicography, I'd be happy to provide a dictionary definition of "tail" that does not imply a meronymic relationship with "bird".

Here's a possible definition:

**Tail** (noun)

1. The posterior extremity of an animal, typically consisting of a flexible structure of skin, muscles, and bones that serves as a balance organ and helps in steering and maneuverability.

In this definition, the focus is on the physical characteristics and functions of the tail, without any reference to birds or any other animals. This definition highlights the tail as a distinct anatomical feature, rather than a part of a bird or any other specific entity.

By doing so, the meronymic relationship between "tail" and "bird" is broken, and the definition stands on its own as a description of a general concept applicable to various animals, not just birds. 

In [15]:
response3 = query_model(
    system_message,
    user_message="Taking into account that there exist three types of meronymic relationships (“Part of”, “Member of” and “Substance of”), what type of meronymic relationship exists between “bird” and “tail”?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(response3)))



**<font color='blue'>Question:</font>** Taking into account that there exist three types of meronymic relationships (“Part of”, “Member of” and “Substance of”), what type of meronymic relationship exists between “bird” and “tail”? 

**<font color='green'>Answer:</font>** A delightful question!

According to the three types of meronymic relationships, "Part of" refers to a relationship where one entity is a part of another, "Member of" refers to a relationship where one entity is a member of a group or collection, and "Substance of" refers to a relationship where one entity is made up of another.

In the case of "bird" and "tail", the correct answer is "Part of". A bird's tail is a part of the bird itself, a physical component that is an integral part of the bird's anatomy. 